In [ ]:
#!pip install crewai
#!pip install crewai_tools
#!pip install langchain_community
#!pip install langchain_openai
#!pip install openai
#!pip install numpy

In [53]:
import os
from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew #Import the Agent, Task and Crew classes from the crewAI libray.
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool #Import crewAI tools that will allow the agent to perform search (google, websites) and scrape websites.

In [62]:
#OpenAI API.
#openai_api_key = 'ADD HERE YOUR API KEY'
#openai_model_name = 'gpt-3.5-turbo'
#openai_api_base = 'https://api.openai.com/v1'
#llm = ChatOpenAI(openai_api_key = openai_api_key, openai_api_base = openai_api_base, model_name = openai_model_name)

#Groq API.
groq_api_key = 'ADD HERE YOUR API KEY'
model_name = 'llama-3.3-70b-versatile'
api_base = 'https://api.groq.com/openai/v1'
llm = ChatOpenAI(openai_api_key = groq_api_key, openai_api_base = api_base, model_name = model_name)

In [55]:
#Create the Support Agent.
support_agent = Agent(
    role = "Senior Support Representative",
	  goal = "Be the most friendly and helpful support representative in your team",
	  backstory = (
	              "You work at crewAI (https://crewai.com) and are now working on providing "
		            "support to {customer}, a super important customer for your company."
		            "You need to make sure that you provide the best support!"
		            "Make sure to provide full complete answers, and make no assumptions."
	              ),
	allow_delegation=False,
	llm = llm,
	verbose = True,
)

In [56]:
#Create the Support Quality Assurance Agent.
support_quality_assurance_agent = Agent(
    role = "Support Quality Assurance Specialist",
	  goal = "Get recognition for providing the best support quality assurance in your team",
	  backstory = (
		            "You work at crewAI (https://crewai.com) and are now working with your team "
		            "on a request from {customer} ensuring that the support representative is providing the best support possible.\n"
		            "You need to make sure that the support representative is providing full complete answers, and make no assumptions."
	              ),
    #allow_delegation=False, #By setting allow_delegation to default TRUE, the agent can delegate its work to another agent, better suited to do a particular task.
	  llm = llm,
		verbose = True,
)

In [57]:
#Scrape one webpage from CreAI documentation.
webpage_scrape_tool = ScrapeWebsiteTool(
    website_url = "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [58]:
#Define the Support Agent task, and assign the webpage_scrape_tool to the task.
inquiry_resolution = Task(
    description = (
                  "{customer} just reached out with a super important ask:\n"
	                "{inquiry}\n\n"
                  "{person} from {customer} is the one that reached out. "
		              "Make sure to use everything you know to provide the best support possible."
		              "You must strive to provide a complete and accurate response to the customer's inquiry."
                  ),
    expected_output = (
	                    "A detailed, informative response to the customer's inquiry that addresses all aspects of their question.\n"
                      "The response should include references to everything you used to find the answer, including external data or solutions. "
                      "Ensure the answer is complete, leaving no questions unanswered, and maintain a helpful and friendly tone throughout."
                      ),
	  tools = [webpage_scrape_tool],
    agent = support_agent,
)

In [59]:
#Define the Support Quality Assurance task. No tool(s) assigned to agent.
quality_assurance_review = Task(
    description = (
                  "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
                  "Ensure that the answer is comprehensive, accurate, and adheres to the high-quality standards expected for customer support.\n"
                  "Verify that all parts of the customer's inquiry have been addressed thoroughly, with a helpful and friendly tone.\n"
                  "Check for references and sources used to find the information, ensuring the response is well-supported and leaves no questions unanswered."
                  ),
    expected_output = (
                      "A final, detailed, and informative response ready to be sent to the customer.\n"
                      "This response should fully address the customer's inquiry, incorporating all relevant feedback and improvements.\n"
		                  "Don't be too formal, we are a chill and cool company but maintain a professional and friendly tone throughout."
                      ),
    agent = support_quality_assurance_agent,
)

In [ ]:
#Create the Crew of agents.
crew = Crew(
    agents = [support_agent, support_quality_assurance_agent],
    tasks = [inquiry_resolution, quality_assurance_review],
    verbose = True,
    memory = True #Enable the short and long term memory.
)

In [61]:
#Define your inputs and trigger the chain of tasks.
inputs = {
    "customer": "DeepLearningAI",
    "person": "Adrian TK",
    "inquiry": "I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs = inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Adrian TK from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
Thought: To provide the best support for DeepLearningAI, I need to find information on how to set up a Crew and kick it off, specifically how to add memory to the Crew. I should start by reading the website content of crewAI to find relevant information.

Action: Read website content
Action Input: {"url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"} 

CrewAI Documentation - CrewAISkip to main contentCrewAI home pageEnglishSearch...⌘KStar